In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.regularizers import l2

Prepare the Dataset

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
filepath_2023 = "/content/gdrive/MyDrive/Colab Notebooks/data/2023 S_S COURSE.xlsx"
filepath_2022 = "/content/gdrive/MyDrive/Colab Notebooks/data/2022 F_S COURSE.xlsx"

In [ ]:
df = pd.read_excel(filepath_2023)
df1 = pd.read_excel(filepath_2022)

In [ ]:
df

,Course Name,Unit,Job Label
0,Political science,Political Science,Diplomat
1,Political science,Political Science,Politician
2,Economics,Economics,Bank Staff
3,Economics,Economics,Economist
4,Economics,Economics,Finance Officer
...,...,...,...
1094,Cultural Creativity Adding Value,Digital Content and Technologies/B/4,Engineer
1095,Computer Animation,Digital Content and Technologies/B/4,Engineer
1096,Interaction Technologies Research and Discussion,Digital Content and Technologies/B/4,Engineer
1097,Artificial Intelligence and Digital Content,Digital Content and Technologies/B/4,Engineer


In [ ]:
df1

,Course Name,Unit,Job Label
0,Macroeconomics,Economics,Economist
1,Macroeconomics,Economics,Supply Chain Manager
2,Economics,Economics,Economist
3,Economics,Economics,Supply Chain Manager
4,Public Finance,Public Finance,Bank staff
...,...,...,...
2747,NaN,NaN,NaN
2748,NaN,NaN,NaN
2749,NaN,NaN,NaN
2750,NaN,NaN,NaN


In [ ]:
df_combined = pd.concat([df[['Job Label', 'Course Name', 'Unit']], df1[['Job Label', 'Course Name', 'Unit']]])

In [ ]:
df_combined['Course Name'] = df_combined['Course Name'].str.lower()
df_combined['Job Label'] = df_combined['Job Label'].str.lower()

In [ ]:
df_combined

,Job Label,Course Name,Unit
0,diplomat,political science,Political Science
1,politician,political science,Political Science
2,bank staff,economics,Economics
3,economist,economics,Economics
4,finance officer,economics,Economics
...,...,...,...
2747,NaN,NaN,NaN
2748,NaN,NaN,NaN
2749,NaN,NaN,NaN
2750,NaN,NaN,NaN


Data Processing

In [ ]:
encoder = LabelEncoder()
df_combined['encoded_job'] = encoder.fit_transform(df_combined['Job Label'])
num_classes = len(encoder.classes_)

In [ ]:
train_df, test_df = train_test_split(df_combined, test_size=0.2, random_state=42)

In [ ]:
train_df['Course Name'] = train_df['Course Name'].astype(str)
test_df['Course Name'] = test_df['Course Name'].astype(str)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Course Name'])
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_df['Course Name'])
test_sequences = tokenizer.texts_to_sequences(test_df['Course Name'])

In [ ]:
max_sequence_length = max(len(seq) for seq in train_sequences)
train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

In [ ]:
num_features = max_sequence_length
num_classes = len(df_combined['Course Name'].unique())

In [ ]:
num_classes = len(encoder.classes_)
train_labels = to_categorical(train_df['encoded_job'], num_classes=num_classes)
test_labels = to_categorical(test_df['encoded_job'], num_classes=num_classes)

Build DNN Model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(LSTM(256, dropout=0.5, recurrent_dropout=0.5, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(LSTM(256, dropout=0.5, recurrent_dropout=0.5, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=20, batch_size=32, validation_data=(test_data, test_labels))

Epoch 1/20
97/97 [==============================] - 42s 320ms/step - loss: 3.4018 - accuracy: 0.3153 - val_loss: 2.7369 - val_accuracy: 0.3463
Epoch 2/20
97/97 [==============================] - 23s 233ms/step - loss: 2.6462 - accuracy: 0.3519 - val_loss: 2.1286 - val_accuracy: 0.4799
Epoch 3/20
97/97 [==============================] - 21s 221ms/step - loss: 2.0450 - accuracy: 0.4870 - val_loss: 1.8598 - val_accuracy: 0.5175
Epoch 4/20
97/97 [==============================] - 23s 238ms/step - loss: 1.8484 - accuracy: 0.5377 - val_loss: 1.7551 - val_accuracy: 0.5460
Epoch 5/20
97/97 [==============================] - 21s 221ms/step - loss: 1.7156 - accuracy: 0.5669 - val_loss: 1.6834 - val_accuracy: 0.5811
Epoch 6/20
97/97 [==============================] - 23s 234ms/step - loss: 1.6640 - accuracy: 0.5701 - val_loss: 1.6629 - val_accuracy: 0.5733
Epoch 7/20
97/97 [==============================] - 24s 244ms/step - loss: 1.5846 - accuracy: 0.5955 - val_loss: 1.6227 - val_accuracy: 0.5759

Evaluate Model

In [ ]:
test_labels = to_categorical(test_df['encoded_job'], num_classes)
loss, accuracy = model.evaluate(test_data, test_labels, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Test Loss: 1.5745317935943604
Test Accuracy: 0.5875486135482788


Recommendation

In [ ]:
unique_job_labels = set(df_combined['Job Label'].str.lower().str.strip()) - {'nan'}

print('=== Job Recommendation Chatbot ===')
print('Hi! Welcome to the Job Recommendation Chatbot.')
print('Please enter one of the following job labels:')

for job_label in unique_job_labels:
    print(job_label)

user_job = input('Enter your job label (or type "exit" to quit): ').lower()

if user_job == 'exit':
    print('Thank you for using the Job Recommendation Chatbot. Goodbye!')
else:
    while True:
        if user_job not in unique_job_labels:
            print('Invalid job label. Please enter a valid job label from the list.')
        else:
            encoded_job = encoder.transform([user_job])
            prediction = model.predict(pad_sequences([encoded_job], maxlen=max_sequence_length))
            predicted_class = prediction.argmax(axis=1)
            recommended_courses = df_combined.loc[df_combined['encoded_job'] == encoded_job[0], ['Course Name', 'Unit']]

            if recommended_courses.empty:
                print('No recommended courses found for the given job label.')
            else:
                random_courses = random.sample(recommended_courses.values.tolist(), k=15)
                print('Recommended Courses:')
                for course, unit in random_courses:
                    print(f'{course} ({unit})')

        user_job = input('Enter your job label (or type "exit" to quit): ').lower()

        if user_job == 'exit':
            print('Thank you for using the Job Recommendation Chatbot. Goodbye!')
            break


=== Job Recommendation Chatbot ===
Hi! Welcome to the Job Recommendation Chatbot.
Please enter one of the following job labels:
project manager
finance officer
human resources generalist
consultant
interpreter
real estate
architect
journalist
sociologist
politician
insurance solicitor
public relations specialist
designer
accountant
lawyer
bureaucrat
historian
bank staff
copywriter
professor
researcher
cyber security
hotel staff
therapist
entrepreneur
supply chain manager
marketer
director
engineer
diplomat
economist
Enter your job label (or type "exit" to quit): director
1/1 [==============================] - 0s 47ms/step
Recommended Courses:
film & tv scriptwriting (Radio & Television/B/3)
media innovation and business model (Radio & Television/B/3)
animation basics and production (Radio & Television/B/3)
news reporting and writing (Undeclared Major in College of Communication/B/1)
audio-visual production basics (Undeclared Major in College of Communication/B/1)
video compression (Com